# Pandas DataFrame-Validierung mit Bulwark

[Bulwark](https://bulwark.readthedocs.io/en/stable/index.html) ist ein Paket zum eigenschaftsbasierten Testen von pandas-Dataframes. Das Projekt wurde stark von der nicht mehr unterstützten [Engarde](https://github.com/engarde-dev/engarde)-Bibliothek beeinflusst.

## 1. Installation

```
$ uv add bulwark
Installing bulwark…
Adding bulwark to Pipfile's [packages]…
✔ Installation Succeeded
Locking [dev-packages] dependencies…
✔ Success!
Updated Pipfile.lock (0d075a)!
```

## 2. Verwendung

### 2.1 Überprüfungen

Mit dem [bulwark.checks](https://bulwark.readthedocs.io/en/v0.4.2/bulwark.html#module-bulwark.checks)-Modul könnt ihr viele gängige Annahmen überprüfen, z.B.

* `has_columns` überprüft, ob bestimmte Spalten so oder so ähnlich vorhanden und in der richtigen Reihenfolge sind
* `has_dtypes` überprüft die Datentypen von Spalten
* `has_no_infs` überprüft, ob keine [numpy.inf](https://numpy.org/doc/stable/reference/constants.html#numpy.inf) im DataFrame vorhanden sind
* `has_no_nans` überprüft, ob es keine [numpy.nan](https://numpy.org/doc/stable/reference/constants.html#numpy.nan) im DataFrame vorhanden sind
* `has_set_within_vals` überprüft, ob die in einem dict angegebenen Werte eine Teilmenge der zugehörigen Spalte sind
* `has_unique_index` überprüft, ob der Index eindeutig ist
* `is_monotonic` überprüft, ob Werte einer Spalte aufsteigend oder absteigend sind
* `one_to_many` überprüft, ob zwischen zwei Spalten eine n:1-Beziehung besteht

Die Überprüfungen sind dann sehr simpel, z.B. der Check, ob in der Spalte `pipe` keine `numpy.nan` vorhanden sind mit
    
```python
import bulwark.checks as ck


df.pipe(ck.has_no_nans())
```



### 2.2 Decorators

Für jeden Check erstellt bulwark.[decorators](https://bulwark.readthedocs.io/en/v0.4.2/bulwark.html#module-bulwark.decorators), z.B. `@dc.IsShape((-1, 10))` oder `@dc.IsMonotonic(strict=True)`.

### `CustomCheck`

Ihr könnt auch eure eigenen benutzerdefinierten Funktionen erstellen, z.B.:

In [1]:
import bulwark.checks as ck
import bulwark.decorators as dc
import numpy as np
import pandas as pd


def len_longer_than(df, l):
    if len(df) <= l:
        raise AssertionError("df is not as long as expected.")
    return df


@dc.CustomCheck(len_longer_than, 10)
def append_a_df(df, df2):
    return pd.concat([df, df2], ignore_index=True)


df = pd.DataFrame({"a": [1, 2, 3], "b": [4, 5, 6]})
df2 = pd.DataFrame({"a": [1, np.nan, 3, 4], "b": [4, 5, 6, 7]})

append_a_df(df, df2)

AssertionError: len_longer_than is not true.

### `MultiCheck`

Mit `MultiCheck` könnt ihr mehrere Tests gleichzeitig ausführen und alle Fehler auf einmal sehen, z.B.:

In [2]:
@dc.MultiCheck(
    checks={ck.has_no_nans: {"columns": None}, len_longer_than: {"l": 6}},
    warn=False,
)
def append_a_df(df, df2):
    return df.append(df2, ignore_index=True)


df = pd.DataFrame({"a": [1, 2, 3], "b": [4, 5, 6]})
df2 = pd.DataFrame({"a": [1, np.nan, 3, 4], "b": [4, 5, 6, 7]})

append_a_df(df, df2)

AttributeError: 'DataFrame' object has no attribute 'append'